# Age and Gender Recognition

### github.com/rafaroman18

This project consist of classifying in real time the gender of a person and their age. We will be using a dataset of faces and a library of python to activate the camera of our laptop.

First of all we will be doing the code necessary to detect faces in our camera. We will be putting a square around every face on our image. 



In [ ]:
# Import
#import numpy as np
import cv2 as cv

cascade_path = ''
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv.VideoCapture(0)


if not cap.isOpened():
    print("Cannot open camera")
    exit()

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        faces = face_cascade.detectMultiScale(
            frame,
            scaleFactor=1.1,
            minNeighbors=5,
            flags=cv.CASCADE_SCALE_IMAGE
        )
        
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the resulting frame
        cv.imshow('frame', frame)
        if cv.waitKey(1) == ord('q'):
            break
finally:
    # Finally, release capture
    cap.release()
    cv.destroyAllWindows()